In [ ]:
import pandas as pd

RAW_FILE = 'search-co-occurrences-in-session.csv'

# query1 query2 count
df = pd.read_csv(RAW_FILE)

df.head()


In [ ]:
import re
def norm_query(q):
    
    
    ### Only accept queries that have at least one char or digit
    ### strip off from everything that is not a char or digit  
    
    ### returns a normalised query string(lowercase, tokens sorted alphabetically and concatenated with '_')
    
    start = 0
    while (start < len(q) and (not (q[start].isalpha() or q[start].isdigit()))):
        start +=1
    
    end = len(q) - 1
    while ((end > start) and (not (q[end].isalpha() or q[end].isdigit()))):
        end -=1
        
    if start >= end:
        return '_____NULL_____' # returning None somehow didn't work here
    
    tokens = re.split("\s+", q[start:end + 1].lower()) 
    tokens.sort()
    return '_'.join(tokens)
    
df['query1_norm'] = pd.Series(df['query1'].apply(norm_query), index=df.index)
df['query2_norm'] = pd.Series(df['query2'].apply(norm_query), index=df.index)

q1 = df[df['query1_norm'] != '_____NULL_____']
queries = q1[q1['query2_norm'] != '_____NULL_____']

# a doc will be a list co-occurring q1, q2 tuples

# trainng docs. (q1,q2) will be added count times, where count is how often the cooccurrence was observed
train_docs = []

for index, row in queries.iterrows():
    q1 = row['query1_norm']
    q2 = row['query2_norm']
    
    if q1 != q2:
        doc = [q1, q2]
        for i in range(row['count']): 
            train_docs.append(doc)
        

    

In [ ]:
#!pip3 install gensim 
import gensim 

print(len(train_docs))

# we could work with less strict settings (VECTOR_SIZE=100, min_counz=1)
VECTOR_SIZE = 300

model = gensim.models.Word2Vec (train_docs, size=VECTOR_SIZE, window=3, min_count=2, workers=10)
model.train(train_docs,total_examples=len(train_docs),epochs=200)



In [ ]:
#model.wv.save('q2vec-d100-c2-e200')
#model.wv.save('q2vec-d300-c1-e100')
#model.wv.save('q2vec-d300-c2-e100')
#model.wv.save('q2vec-d300-c2-e200')
#model.wv.save('q2vec-d300-c2-e400')

In [ ]:
model.wv.most_similar('blue boots')